In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as urllib2
from bs4 import BeautifulSoup
import json

ImportError: Missing required dependencies ['numpy']

In [5]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "<h1>Not Much Going On Here</h1>"
app.run(host='0.0.0.0', port=50000)

IndentationError: unexpected indent (<ipython-input-5-be9e6ab3e984>, line 6)

In [2]:
df = pd.read_csv('dataset_cleaned.csv')

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
y = df['Value']
df.shape

(6762, 34)

In [4]:
X = df.drop(['Value'], axis=1)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1234)

In [6]:
X_train.dtypes

Unnamed: 0        int64
Month             int64
Hour              int64
DIR             float64
SPD             float64
TEMP            float64
Value_1         float64
TEMP_1          float64
SPD_1           float64
DIR_1           float64
Value_2         float64
TEMP_2          float64
SPD_2           float64
DIR_2           float64
Value_3         float64
TEMP_3          float64
SPD_3           float64
DIR_3           float64
Value_4         float64
TEMP_4          float64
SPD_4           float64
DIR_4           float64
Value_5         float64
TEMP_5          float64
SPD_5           float64
DIR_5           float64
HUM             float64
HUM_1           float64
HUM_2           float64
HUM_3           float64
HUM_4           float64
HUM_5           float64
day_week_cat      int64
dtype: object

In [7]:
from xgboost.sklearn import XGBRegressor

xboost = XGBRegressor(n_estimators=200)

In [8]:
xboost.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
xgb_score = xboost.score(X_test, y_test)
xgb_score

0.7432749301007617

In [10]:
X_train.head()

,Month,Hour,DIR,SPD,TEMP,Value_1,TEMP_1,SPD_1,DIR_1,Value_2,...,TEMP_5,SPD_5,DIR_5,HUM,HUM_1,HUM_2,HUM_3,HUM_4,HUM_5,day_week_cat
4808,6,16,350.0,20.92142,7.222222,22.0,7.777778,6.43736,260.0,128.0,...,15.000000,35.40548,990.0,60.131692,57.894632,62.991173,54.289142,58.876705,43.806617,1
4431,5,9,310.0,40.23350,18.888889,40.0,22.777778,28.96812,990.0,30.0,...,21.111111,14.48406,990.0,25.873111,14.017293,10.577524,8.121697,10.366566,10.508683,1
3350,12,0,70.0,6.43736,-30.000000,154.0,-32.222222,3.21868,990.0,122.0,...,-27.777778,3.21868,990.0,76.672049,68.259067,68.807200,72.886260,69.076428,65.736441,1
5469,8,8,360.0,11.26538,27.222222,8.0,26.111111,25.74944,990.0,5.0,...,21.111111,6.43736,250.0,29.266212,30.082759,33.689548,33.376028,51.356790,52.831566,3
3507,1,18,350.0,17.70274,-15.000000,166.0,-15.000000,6.43736,990.0,95.0,...,-13.888889,3.21868,990.0,65.632583,72.187947,72.187947,66.171527,72.417864,72.417864,0


In [11]:
import datetime
import requests
import time

In [12]:
X_train.head(3)

,Month,Hour,DIR,SPD,TEMP,Value_1,TEMP_1,SPD_1,DIR_1,Value_2,...,TEMP_5,SPD_5,DIR_5,HUM,HUM_1,HUM_2,HUM_3,HUM_4,HUM_5,day_week_cat
4808,6,16,350.0,20.92142,7.222222,22.0,7.777778,6.43736,260.0,128.0,...,15.000000,35.40548,990.0,60.131692,57.894632,62.991173,54.289142,58.876705,43.806617,1
4431,5,9,310.0,40.23350,18.888889,40.0,22.777778,28.96812,990.0,30.0,...,21.111111,14.48406,990.0,25.873111,14.017293,10.577524,8.121697,10.366566,10.508683,1
3350,12,0,70.0,6.43736,-30.000000,154.0,-32.222222,3.21868,990.0,122.0,...,-27.777778,3.21868,990.0,76.672049,68.259067,68.807200,72.886260,69.076428,65.736441,1


In [14]:
response = requests.get("https://api.openweathermap.org/data/2.5/weather?q=Ulaanbaatar,MN&appid=0dfd5dd923d46c1cfb6464650f9d7dfc")
a = response.content

In [13]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM_5 = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD_5 = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR_5 = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP_5 = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value_5 = j['results'][10]['value']



In [16]:
#time.sleep(3600)

In [14]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM_4 = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD_4 = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR_4 = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP_4 = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value_4 = j['results'][10]['value']



In [18]:
#time.sleep(3600)

In [15]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM_3 = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD_3 = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR_3 = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP_3 = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value_3 = j['results'][10]['value']



In [20]:
#time.sleep(3600)

In [16]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM_2 = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD_2 = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR_2 = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP_2 = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value_2 = j['results'][10]['value']



In [22]:
#time.sleep(3600)

In [20]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM_1 = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD_1 = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR_1 = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP_1 = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value_1 = j['results'][10]['value']



In [24]:
#time.sleep(3600)

In [21]:
page = urllib2.urlopen("https://weather-details.000webhostapp.com/det.php")
soup = BeautifulSoup(page, 'html.parser')
day_week_cat = 2
HUM = float(soup.find('h1', attrs={'class': 'humidity'}).text.strip()[10:(soup.find('h1', attrs={'class': 'humidity'}).text.strip()).find("%")])
SPD = float(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()[13:(soup.find('h1', attrs={'class': 'wind_speed'}).text.strip()).find("km/hr")])
DIR = float(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()[16:(soup.find('h1', attrs={'class': 'wind_direction'}).text.strip()).find("`")])
TEMP = float(soup.find('h1', attrs={'class': 'temperature'}).text.strip()[13:(soup.find('h1', attrs={'class': 'temperature'}).text.strip()).find("`C")])

response = requests.get("https://api.openaq.org/v1/measurements")
j = json.loads(response.content)
Value = j['results'][10]['value']


Hour = int((soup.find('h1', attrs={'class': 'time'}).text.strip()[7:9]))
Month = 2

In [11]:
Value

157

In [53]:
new_one = pd.DataFrame([[Month,Hour,DIR,SPD,TEMP,Value_1,TEMP_1,SPD_1,DIR_1,Value_2,TEMP_2,SPD_2,DIR_2,Value_3,TEMP_3,SPD_3,DIR_3,Value_4,TEMP_4,SPD_4,DIR_4,Value_5,TEMP_5,SPD_5,DIR_5,HUM,HUM_1,HUM_2,HUM_3,HUM_4,HUM_5,day_week_cat]], columns = ['Month','Hour','DIR','SPD','TEMP','Value_1','TEMP_1','SPD_1','DIR_1','Value_2','TEMP_2','SPD_2','DIR_2','Value_3','TEMP_3','SPD_3','DIR_3','Value_4','TEMP_4','SPD_4','DIR_4','Value_5','TEMP_5','SPD_5','DIR_5','HUM','HUM_1','HUM_2','HUM_3','HUM_4','HUM_5','day_week_cat']) 
new_one

,Month,Hour,DIR,SPD,TEMP,Value_1,TEMP_1,SPD_1,DIR_1,Value_2,...,TEMP_5,SPD_5,DIR_5,HUM,HUM_1,HUM_2,HUM_3,HUM_4,HUM_5,day_week_cat
0,2,11,0.0,2.0,-9.0,110,-12.0,2.0,0.0,150,...,-20.0,0.0,0.0,52.0,66.0,77.0,77.0,84.0,84.0,2


In [54]:
xboost.predict(new_one)

array([141.77774], dtype=float32)

In [31]:
X_test.dtypes

Month             int64
Hour              int64
DIR             float64
SPD             float64
TEMP            float64
Value_1         float64
TEMP_1          float64
SPD_1           float64
DIR_1           float64
Value_2         float64
TEMP_2          float64
SPD_2           float64
DIR_2           float64
Value_3         float64
TEMP_3          float64
SPD_3           float64
DIR_3           float64
Value_4         float64
TEMP_4          float64
SPD_4           float64
DIR_4           float64
Value_5         float64
TEMP_5          float64
SPD_5           float64
DIR_5           float64
HUM             float64
HUM_1           float64
HUM_2           float64
HUM_3           float64
HUM_4           float64
HUM_5           float64
day_week_cat      int64
dtype: object

In [34]:
new_one.dtypes

Month             int64
Hour              int64
DIR             float64
SPD             float64
TEMP            float64
Value_1           int64
TEMP_1          float64
SPD_1           float64
DIR_1           float64
Value_2           int64
TEMP_2          float64
SPD_2           float64
DIR_2           float64
Value_3           int64
TEMP_3          float64
SPD_3           float64
DIR_3           float64
Value_4           int64
TEMP_4          float64
SPD_4           float64
DIR_4           float64
Value_5           int64
TEMP_5          float64
SPD_5           float64
DIR_5           float64
HUM             float64
HUM_1           float64
HUM_2           float64
HUM_3           float64
HUM_4           float64
HUM_5           float64
day_week_cat      int64
dtype: object